In [3]:
import pandas as pd
import pickle
import os
import sys
import numpy as np
import cv2
import torch
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from shapely.geometry import Polygon
from skimage import transform
from torch.utils.data import DataLoader
import torch.nn.functional as F


from floortrans.models import get_model
from floortrans.loaders import (
    FloorplanSVG,
    DictToTensor,
    Compose,
    RotateNTurns
)
from floortrans.plotting import (
    segmentation_plot,
    polygons_to_image,
    draw_junction_from_dict,
    discrete_cmap
)
from floortrans.post_prosessing import (
    split_prediction,
    get_polygons,
    split_validation
)
from mpl_toolkits.axes_grid1 import AxesGrid

%matplotlib inline
from skimage import transform
from matplotlib.pyplot import imshow, show
from matplotlib.ticker import PercentFormatter
from skimage.measure import label as label_func
import time
import random

#discrete_cmap()

os.environ['PYTHONPATH'] = '/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git:' + os.environ.get('PYTHONPATH', '')

rot = RotateNTurns() 

#12 different values for room_classes
room_classes = ["Background", 
                "Outdoor", 
                "Wall", 
                "Kitchen", 
                "Living Room" ,
                "Bed Room", 
                "Bath", 
                "Entry", 
                "Railing", 
                "Storage", 
                "Garage", 
                "Undefined"]
#11 different values for icon_classes
icon_classes = ["No Icon", 
                "Window", 
                "Door", 
                "Closet", 
                "Electrical Applience" ,
                "Toilet", 
                "Sink", 
                "Sauna Bench", 
                "Fire Place", 
                "Bathtub", 
                "Chimney"]

data_folder = '../data/cubicasa5k/'
#change to train.txt, val.txt, or test.txt
data_file = 'test.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)
# Setup Model
model = get_model('hg_furukawa_original', 51)

n_classes = 44
split = [21, 12, 11]
print("Model loaded")



Model loaded


In [5]:
random.seed(1)

X = []
Y = []
image_idx=[]
errors = pd.DataFrame(columns=['Index', 'Room'])
def isolate_class(rooms, CLASS: int):
    template = np.zeros_like(rooms)
    rows, cols = np.where(rooms == CLASS)
    template[rows, cols] = 1
    return template

for i, val in enumerate(data_iter):
    junctions = val['heatmaps']
    folder = val['folder'][0]
    image = val['image']
    label = val['label']
    label_np = label.data.numpy()[0]
    rooms=label_np[0].astype(np.uint8)
    icons=label_np[1].astype(np.uint8)
    for room in range(len(room_classes)):
        y = room # looping room by room
        isolated_room = isolate_class(rooms, room) # selects the region on the image where the class exists
        room_contours, _ = cv2.findContours(isolated_room, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # find individual incidents of the class, RETR_EXTERNAL just means only consider external contours (no donuts)
        for rc in room_contours: # loop through each individual incidents
            rc = np.squeeze(rc, 1) # removes the first dimension
            if rc.shape[0]>=4:
                room_cont = Polygon(rc).buffer(0) # the .buffer(0) operation is the simpliest way to ensure that the polygon stays valid
                x = np.zeros(len(icon_classes)) # initiate a new x incident
                for icon in range(len(icon_classes)): # loop through all the icons
                    isolated_icon = isolate_class(icons, icon) # selects the region on the image where the class exists
                    icon_contours, _ = cv2.findContours(isolated_icon, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # find indivudual incidents of the class
                    for ic in icon_contours: # loop through each individual incidents
                        ic = np.squeeze(ic, 1) # removes the first dimension
                        
                        if ic.shape[0]>=4:
                            icon_cont = Polygon(ic).buffer(0) # again...
                            if room_cont.intersects(icon_cont): # <- if they intersects then theres one incident of overlapping between the room type and icon type
                                x[icon] += 1 # for this room type incident, there's one more incident of overlap with this icon class
                X.append(x)
                Y.append(y)
                image_idx.append(i)
            else:
                errors.loc[0 if pd.isnull(errors.index.max()) else errors.index.max() + 1]=[i, room]   
    print(f"{i+1} done", end="\r", flush=True)

data=np.vstack(X)
df=pd.DataFrame(data,columns=icon_classes)
df['Image_idx']=image_idx
df['Room']=Y

In [6]:
df

,No Icon,Window,Door,Closet,Electrical Applience,Toilet,Sink,Sauna Bench,Fire Place,Bathtub,Chimney,Image_idx,Room
0,1.0,16.0,12.0,3.0,6.0,2.0,2.0,1.0,1.0,0.0,0.0,0,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6162,1.0,0.0,0.0,2.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,379,7
6163,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,379,8
6164,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,379,8
6165,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,379,8
